# Assess RNA binding re-parameterisation

The RNA binding data from paper ["Metabolic engineering of Escherichia coli using synthetic small regulatory RNAs"](https://www.nature.com/articles/nbt.2461#Fig1) 

In [2]:
from copy import deepcopy
from functools import partial
import os
import sys

import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('bmh')
plt.style.use('seaborn-v0_8')
import numpy as np
import jax.numpy as jnp
import jax.random as jr
import pandas as pd
import jax
import diffrax as dfx
from tqdm import tqdm

# os.environ["XLA_PYTHON_CLIENT_PREALLOCATE"] = "false"
# os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"] = "0.8"
os.environ["TF_CPP_MIN_LOG_LOVEL"] = "0"
jax.config.update('jax_platform_name', 'cpu')

import bioreaction
from bioreaction.model.data_containers import Impulse
from bioreaction.simulation.basic_sim import basic_de_sim, convert_model, BasicSimParams, BasicSimState, bioreaction_sim_expanded
from bioreaction.model.data_tools import construct_model_fromnames
from bioreaction.simulation.med_sim import get_int_med_model, basic_de, MedSimParams, MedSimInternalState


if __package__ is None:

    module_path = os.path.abspath(os.path.join('..'))
    sys.path.append(module_path)

    __package__ = os.path.basename(module_path)

from tests.shared import five_circuits, CONFIG
import src
from src.srv.sequence_exploration.sequence_analysis import b_tabulate_mutation_info
from src.utils.misc.numerical import make_symmetrical_matrix_from_sequence, triangular_sequence
from src.utils.misc.type_handling import flatten_listlike
from src.utils.misc.units import per_mol_to_per_molecule
from src.utils.modelling.deterministic import bioreaction_sim_dfx_expanded
from src.utils.results.analytics.naming import get_true_names_analytics, get_true_interaction_cols
from src.utils.results.analytics.timeseries import generate_analytics, get_precision, get_sensitivity


config = deepcopy(CONFIG)

jax.devices()

xla_bridge.py:backends():355: Unable to initialize backend 'tpu_driver': NOT_FOUND: Unable to find driver in registry given worker:  INFO
xla_bridge.py:backends():355: Unable to initialize backend 'rocm': NOT_FOUND: Could not find registered platform with name: "rocm". Available platform names are: Host Interpreter CUDA INFO
xla_bridge.py:backends():355: Unable to initialize backend 'tpu': module 'jaxlib.xla_extension' has no attribute 'get_tpu_client' INFO
xla_bridge.py:backends():355: Unable to initialize backend 'plugin': xla_extension has no attributes named get_plugin_device_client. Compile TensorFlow with //tensorflow/compiler/xla/python:enable_plugin_device set to true (defaults to false) to enable this. INFO


[CpuDevice(id=0)]

Examining previously generated RNA sequences for which the $\Delta G$ has been simulated and translated into 

In [3]:
path_to_stats = 'data/ensemble_generate_circuits/2023_02_24_170946/gather_interaction_stats/circuit_stats.csv'
path_to_stats = os.path.join('..', path_to_stats)

circuit_stats = pd.read_csv(path_to_stats)

In [4]:
circuit_stats.columns

Index(['name', 'interacting', 'self_interacting', 'num_interacting',
       'num_self_interacting', 'binding_sites_0-0', 'binding_sites_0-1',
       'binding_sites_0-2', 'binding_sites_1-0', 'binding_sites_1-1',
       'binding_sites_1-2', 'binding_sites_2-0', 'binding_sites_2-1',
       'binding_sites_2-2', 'binding_rates_dissociation_0-0',
       'binding_rates_dissociation_0-1', 'binding_rates_dissociation_0-2',
       'binding_rates_dissociation_1-0', 'binding_rates_dissociation_1-1',
       'binding_rates_dissociation_1-2', 'binding_rates_dissociation_2-0',
       'binding_rates_dissociation_2-1', 'binding_rates_dissociation_2-2',
       'energies_0-0', 'energies_0-1', 'energies_0-2', 'energies_1-0',
       'energies_1-1', 'energies_1-2', 'energies_2-0', 'energies_2-1',
       'energies_2-2', 'eqconstants_0-0', 'eqconstants_0-1', 'eqconstants_0-2',
       'eqconstants_1-0', 'eqconstants_1-1', 'eqconstants_1-2',
       'eqconstants_2-0', 'eqconstants_2-1', 'eqconstants_2-2',
      

In [8]:
cols_energies = [c for c in circuit_stats.columns if 'energies' in c and ('path' not in c)]
cols_eqconstants = [c for c in circuit_stats.columns if 'eqconstants' in c and ('path' not in c)]


In [10]:
dfm = circuit_stats.melt(id_vars=['name', 'num_interacting', 'num_self_interacting'],
                         value_vars=cols_energies, value_name='energies')
dfm['eqconstants_gibbs'] = dfm1['eqconstants']
dfm


,name,num_interacting,num_self_interacting,variable,energies,eqconstants
0,toy_mRNA_circuit_0,0,0,energies_0-0,30.00,0.000009
1,toy_mRNA_circuit_100000,0,0,energies_0-0,30.00,0.000009
2,toy_mRNA_circuit_100001,0,0,energies_0-0,30.00,0.000009
3,toy_mRNA_circuit_100002,1,0,energies_0-0,30.00,0.000009
4,toy_mRNA_circuit_100003,0,0,energies_0-0,30.00,0.000009
...,...,...,...,...,...,...
1799995,toy_mRNA_circuit_99999,0,0,energies_2-2,30.00,0.000009
1799996,toy_mRNA_circuit_9999,0,1,energies_2-2,30.00,0.000009
1799997,toy_mRNA_circuit_999,0,1,energies_2-2,30.00,0.000009
1799998,toy_mRNA_circuit_99,0,0,energies_2-2,30.00,0.000009
